In [1]:
import numpy as np
import geotable
import networkx as nx
import json
import matplotlib.pyplot as plt
import random
import pyproj
import geojson
from haversine import haversine, Unit
import geopy

import utm
import numpy as np
import numpy.linalg as LA
import random
import math
import time

import pandas as pd
import pydeck as pdk

import pydeck as pdk
import pandas as pd
import ipywidgets
from copy import deepcopy
# use palettable for colormap colors
from palettable.cartocolors.sequential import BrwnYl_3
import json
from math import radians, cos, sin, asin, sqrt

Colors_RGB = [[31, 119, 180],
              [255, 127, 14],
              [44, 160, 44],
              [214, 39, 40],
              [148, 103, 189],
              [140, 86, 75],
              [227, 119, 194],
              [127, 127, 127],
              [188, 189, 34],
              [23, 190, 207]]

    
JFK_LAT_LON = 40.645944, -73.784839



LL = 39.88494891-1, -75.33930212-1
UR = 41.888126+1, -72.73301048+3
geo_locator = geopy.Nominatim(user_agent='12334')

ModuleNotFoundError: No module named 'osgeo'

In [ ]:
g = nx.Graph()
vertiport_locations = geotable.load("../Vertiport_Locations/Vertiport_Locations.kmz")

In [ ]:

def is_in(lat,lon,UR,LL):
    return (lat >= LL[0]) and (lat <= UR[0]) and (lon >= LL[1]) and (lon <= UR[1])

def ccw(A,B,C):
    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

# Return true if line segments AB and CD intersect
def intersect(A,B,C,D):
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371000 # Radius of earth in meters. Use 3956 for miles. Determines return value units.
    return c * r

def angle_of_vectors(a,b):


    inner = np.inner(a, b)
    norms = LA.norm(a) * LA.norm(b)

    cos = inner / norms
    rad = np.arccos(np.clip(cos, -1.0, 1.0))
    deg = np.rad2deg(rad)
    
    return deg



In [ ]:
Num_Vertiports = len(vertiport_locations)

lons = []
lats = []
Xs = []
Ys = []
New_Long_Lat = {}
i = 0

for i in range(Num_Vertiports):
    
    lon = vertiport_locations["geometry_object"][i].x
    lat = vertiport_locations["geometry_object"][i].y
    
    x, y, z, zl = utm.from_latlon(lat, lon)
    
    lons.append(lon)
    lats.append(lat)

    Xs.append(x)
    Ys.append(y)
    g.add_node(i)
    
    
    New_Long_Lat[i] = [lon, lat]
    
    i += 1
    
    

In [ ]:
distmx = np.zeros((len(New_Long_Lat), len(New_Long_Lat)))
for i in range(Num_Vertiports):
    for j in range(Num_Vertiports):
        dist = ((lons[i]-lons[j])**2 + (lats[i]-lats[j])**2)**0.5
        
        distmx[i,j] = dist
        g.add_edge(i,j)

In [ ]:
SRC_list = []
DST_list = []

for e in g.edges:
    SRC_list.append([lons[int(e[0])], lats[int(e[0])]])
    DST_list.append([lons[int(e[1])], lats[int(e[1])]])
    
dictv = {'start': SRC_list, 'end': DST_list} 

edge_df = pd.DataFrame(dictv)
    

In [ ]:
full_df = {}


Node_Lon_Lat = []
Node_Color = []
Size = []
Vertiport = []
Tooltip = []


for i in range(len(lons)):

    if (i < Num_Vertiports):
        Node_Lon_Lat.append([lons[i], lats[i]])
        Node_Color.append([0, 255, 0, 255])
        Size.append(10000)
        Tooltip.append(f"<b>Type:</b> Vertiport: {i}")

    if (i >= Num_Vertiports):

        Node_Lon_Lat.append([lons[i], lats[i]])
        Node_Color.append([0, 255, 0, 50])
        Size.append(1.5)
        Tooltip.append(f"<b>Type:</b> Node: {i}")


dictv = {'coordinates': Node_Lon_Lat, 'color': Node_Color, 'size': Size, 'tooltip': Tooltip} 

df = pd.DataFrame(dictv)


In [ ]:

sunlight = {
    "@@type": "_SunLight",
    "timestamp": 1564696800000,  # Date.UTC(2019, 7, 1, 22),
    "color": [255, 255, 255],
    "intensity": 1.0,
    "_shadow": True,
}

ambient_light = {"@@type": "AmbientLight", "color": [255, 255, 255], "intensity": 1.0}

lighting_effect = {
    "@@type": "LightingEffect",
    "shadowColor": [0, 0, 0, 0.5],
    "ambientLight": ambient_light,
    "directionalLights": [sunlight],
}

view_state = pdk.ViewState(
    **{"latitude": JFK_LAT_LON[0], "longitude": JFK_LAT_LON[1], "zoom": 10, "maxZoom": 14, "pitch": 90, "bearing": 0}
)




layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    radius_scale=0.1,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius="size",
    get_fill_color="color",
    get_line_color=[0, 0, 0],
)

line_layer = pdk.Layer(
    "LineLayer",
    edge_df,
    get_source_position="start",
    get_target_position="end",
    get_color=[105,105,105],
    get_width=0.5,
    highlight_color=[255, 255, 0],
    picking_radius=10,
    auto_highlight=True,
)

tooltip = {"html": "{tooltip}"}

keys = {"mapbox": "pk.eyJ1IjoibWNkb25zdCIsImEiOiJjbDRyNGphOWwweHA1M2lxemI1ZXhmM3Y1In0.Km2Tt6dKpTqwfNmTMEKrQA"}


layers = [line_layer, layer]

r = pdk.Deck(
    layers,
    initial_view_state=view_state,
    effects=[lighting_effect],
    map_style="dark_no_labels",
    api_keys=keys,
    tooltip=tooltip,
)




# # display and save map (to_html(), show())
r.show()

In [ ]:
Adj = np.zeros((len(lons),len(lons)))

In [ ]:
for e in g.edges:
    
    lon1 = New_Long_Lat[e[0]][0]
    lat1 = New_Long_Lat[e[0]][1]
    
    lon2 = New_Long_Lat[e[1]][0]
    lat2 = New_Long_Lat[e[1]][1]
    
    dist = haversine(lon1, lat1, lon2, lat2)
    
    Adj[e[0],e[1]] = dist
    Adj[e[1],e[0]] = dist

In [ ]:

with open('LongLat.json', 'w') as f:
    json.dump(New_Long_Lat, f)
    
nx.set_node_attributes(g, New_Long_Lat, name="pos")

Verts = {}
for i in range(len(lons)):
    if i < Num_Vertiports:
        Verts[i] = 1
    else:
        Verts[i] = 0
        
nx.set_node_attributes(g, Verts, name="vertiport")

In [ ]:
Vert_Zip = {}

for n in range(Num_Vertiports):
    
    if n == 10:
        Vert_Zip[10] = int("06511")
        continue
    
    r = geo_locator.reverse((New_Long_Lat[n][1], New_Long_Lat[n][0]))
    zip_code = r.raw['address']['postcode']
    
    Vert_Zip[n] = int(zip_code)
    
with open('Vert_Zip.json', 'w') as f:
    json.dump(Vert_Zip, f)

In [ ]:
nx.write_gml(g, "Graph.gml")

In [ ]:
Edges = {}
k = 1
for e in g.edges:
    Edges[k] = [e[0]+1, e[1]+1, Adj[e[0],e[1]]]
    k += 1
    
Edges["Nodes"] = g.number_of_nodes()
Edges["Edges"] = g.number_of_edges()
Edges["Number_of_Vertiports"] = Num_Vertiports


In [ ]:
with open('Edges.json', 'w') as f:
    json.dump(Edges, f)